In [1]:
import os
import sys

os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable

In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
#from gristmill import *
 


In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[*]', 'bcs_ccsd')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 14:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

finished


In [4]:
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N

In [23]:
P_i_dag = (u[p]*v[p]+u[p]**2*Pdag[p]
     - u[p]*v[p]*N[p]
     - v[p]**2*P[p])

P_j_dag = (
        u[q]*v[q]
      + u[q]**2 * Pdag[q]
      - u[q]*v[q] * N[q]
      - v[q]**2 * P[q]
    )

P_k_dag = (
        u[r]*v[r]
      + u[r]**2 * Pdag[r]
      - u[r]*v[r] * N[r]
      - v[r]**2 * P[r]
    )
P_i = (
        (u[p])*(v[p])
      + (u[p])**2 * P[p]
      - (u[p])*(v[p]) * N[p]
      - (v[p])**2 * Pdag[p]
    )
P_j = ((u[q])*(v[q])
      + (u[q])**2 * P[q]
      - (u[q])*(v[q]) * N[q]
      - (v[q])**2 * Pdag[q])

P_k = ((u[r])*(v[r])
      + (u[r])**2 * P[r]
      - (u[r])*(v[r]) * N[r]
      - (v[r])**2 * Pdag[r])
#hc = P_j_dag * P_i
N_i = 2*(v[p])*v[p] +((u[p])*u[p] - (v[p])*v[p])*N[p] +2*u[p]*(v[p])*Pdag[p]\
            +2*(u[p])*v[p]*P[p]
N_j = 2*(v[q])*v[q] +((u[q])*u[q] - (v[q])*v[q])*N[q] +2*u[q]*(v[q])*Pdag[q]\
            +2*(u[q])*v[q]*P[q]

N_k = 2*(v[r])*v[r] +((u[r])*u[r] - (v[r])*v[r])*N[r] +2*u[r]*(v[r])*Pdag[r]\
       +2*(u[r])*v[r]*P[r]


In [5]:
p,q,r,s,i,j,k,l = dr.names.A_dumms[:8]

In [6]:
u, v = IndexedBase('u'), IndexedBase('v')
E0 = Symbol(r'G')
H120, H021 = IndexedBase('h120') , IndexedBase('h021')
H011, H110 = IndexedBase('h011'), IndexedBase('h110')
H100, H011 = IndexedBase('h100'), IndexedBase('h011')
H001, H020 = IndexedBase('h001'), IndexedBase('h020')
H010 = IndexedBase('h010') 
H000 = IndexedBase('h000')
H11, H20 , H02 = IndexedBase('H11'), IndexedBase('H20'), IndexedBase('H02')
H04, H40 = IndexedBase('H04'), IndexedBase('H40')
H22, Hb22 = IndexedBase('H22'), IndexedBase('Hb22')
H31, H13 = IndexedBase('H31'), IndexedBase('H13')

In [7]:
expr = H11[p]*N[p] + H02[p]*Pdag[p] +H20[p]*P[p]+H22[p,q]*N[p]*N[q]+Hb22[p,q]*Pdag[p]*P[q]\
    +H40[p,q]*P[p]*P[q]+H04[p,q]*Pdag[p]*Pdag[q]+H13[p,q]*Pdag[p]*N[q]+H31[p,q]*N[p]*P[q]
H_N = dr.einst(expr).simplify().merge()


In [8]:
H_N.display()

<IPython.core.display.Math object>

In [9]:
t1, t2, t3 = IndexedBase('t1'), IndexedBase('t2'), IndexedBase('t3')

z1, z2 = IndexedBase('z1'), IndexedBase('z2')

Z1 = dr.einst(z1[p]*P[p])
Z2 = dr.einst(Rational(1,2)*z2[p,q]*P[p]*P[q])

T1 = dr.einst(t1[p] * Pdag[p])
T2 = dr.einst(Rational(1, 2) * t2[p,q] *Pdag[p]*Pdag[q])
T3 = dr.einst(Rational(1,6)*t3[p,q,r]*Pdag[p]*Pdag[q]*Pdag[r])

Z = Z1 + Z2
cluster = T1 + T2 #+ T3
Z.display()

<IPython.core.display.Math object>

In [10]:
dr.set_symm(t2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(z2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(H04,Perm([1,0],IDENT))
dr.set_symm(H40,Perm([1,0],IDENT)) 
#dr.set_symm(t3, Perm([1,0,2],IDENT),Perm([2,1,0],IDENT), Perm([0,2,1],IDENT), valence=3)
print("Perm works!")

Perm works!


In [11]:
zero_term = [(H40[p,p],0),(H04[p,p],0),(t2[p,p],0),(z2[p,p],0)]


In [12]:
Hbar = H_N
curr = H_N
fact = 1
for n in range(4):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    Hbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("Hbar evaluation done")

[Stage 117:=====================>                                   (3 + 5) / 8]

Hbar evaluation done


In [13]:
E_eqn  = Hbar.simplify().eval_vev().simplify()
E_eqn = E_eqn.subst_all(zero_term)
#print(E_eqn.latex())
E_eqn.display()


<IPython.core.display.Math object>

In [14]:
s1_eqn_CCSD = (P[p] * Hbar).simplify().eval_vev().simplify()
print('evaluation done')

evaluation done


In [15]:
s1_eqn_CCSD = s1_eqn_CCSD.subst_all(zero_term)
print(s1_eqn_CCSD.latex())

- t^{(1)}_{p}^{2}  H^{(20)}_{p}  - 2 t^{(1)}_{p}^{2}  H^{(31)}_{p,p}   + 2 H^{(11)}_{p}  t^{(1)}_{p}   + 4 t^{(1)}_{p}  H^{(22)}_{p,p}   + H^{(02)}_{p}   + \sum_{q \in A} H^{(20)}_{q}  t^{(2)}_{q,p}   + \sum_{q \in A} t^{(1)}_{q}  Hb^{(22)}_{p,q}   + \sum_{q \in A} 2 H^{(31)}_{p,q}  t^{(2)}_{q,p}  - \sum_{q \in A} 4 t^{(1)}_{p}  H^{(40)}_{q,p}  t^{(2)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{p}^{2}  t^{(1)}_{q}  H^{(40)}_{q,p}   + \sum_{q \in A} 2 t^{(1)}_{p}  t^{(1)}_{q}  H^{(31)}_{p,q}   + \sum_{q \in A} \sum_{r \in A} 2 t^{(1)}_{q}  H^{(40)}_{q,r}  t^{(2)}_{r,p} 


In [16]:
s2_eqn_CCSD = (P[p]*P[q] * Hbar).simplify().eval_vev().simplify()
s2_eqn_CCSD = s2_eqn_CCSD.subst_all(zero_term)
print('evaluation done')

evaluation done


In [17]:
def drop_delta_terms_str(tens: Tensor) -> Tensor:
    # Remove any term whose printed form contains 'KroneckerDelta('
    filt = tens.terms.filter(lambda term: 'KroneckerDelta' not in str(term))
    return Tensor(tens._drudge, filt).merge().simplify()

s1_eqn_CCSD = drop_delta_terms_str(s1_eqn_CCSD)
s2_eqn_CCSD = drop_delta_terms_str(s2_eqn_CCSD)

In [18]:
'''Defining CC Lagrangian
L = <0|(1+Z) bar{H}|0> and dL/dt = 0 is the Z equations. dL/dt = <0|(1+Z)[bar{H},Q_{mu}]|0>
'''

#dL_dt_1 = (1+Z)*(Hbar|Pdag[p]).simplify().eval_vev().simplify().merge()
dL_dt_1 = ((1+Z)*(Hbar|Pdag[p])).simplify().eval_vev().simplify()

In [20]:
dL_dt_2 = ((1+Z)*(Hbar|Pdag[p]*Pdag[q])).simplify().eval_vev().simplify()
print('finish')

[Stage 2304:===================================>                    (5 + 3) / 8]

finish


In [21]:
dL_dt_2 = dL_dt_2.subst_all(zero_term)
print(dL_dt_2.latex())

2 H^{(40)}_{p,q}   + H^{(20)}_{p}  z^{(1)}_{q}   + H^{(20)}_{q}  z^{(1)}_{p}   + 2 H^{(11)}_{p}  z^{(2)}_{p,q}   + 2 H^{(11)}_{q}  z^{(2)}_{p,q}   + 2 z^{(1)}_{p}  H^{(31)}_{p,q}   + 2 z^{(1)}_{q}  H^{(31)}_{q,p}   + 4 H^{(22)}_{p,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{p,q}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  z^{(1)}_{p}  H^{(40)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{p,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{q,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  z^{(1)}_{q}  H^{(40)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{p,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{q,q}  z^{(2)}_{p,q}  -  4 \delta_{p q} z^{(1)}_{p}  H^{(31)}_{p,p}  - 2 H^{(20)}_{p}  t^{(1)}_{p}  z^{(2)}_{p,q}  -  2 \delta_{p q} H^{(20)}_{p}  z^{(1)}_{p}  - 2 H^{(20)}_{q}  t^{(1)}_{q}  z^{(2)}_{p,q}   + 8 H^{(40)}_{p,q}  t^{(2)}_{p,q}  z^{(2)}_{p,q}   + 8 t^{(1)}_{p}  t^{(1)}_{q}  H^{(40)}_{p,q}  z^{(2)}_{p,q}   + \sum_{r \in A} Hb^{(22)}_{r,p}  z^{(2)}_{r,q}   + \sum_{r 

In [24]:
RDM1 = ((1+Z)*(P_i_dag)).simplify().eval_vev().simplify()

In [25]:
print(RDM1.latex())

u_{p}^{2}  z^{(1)}_{p}   + u_{p}  v_{p} 


In [29]:
RDM2 = ((1+Z)*(P_i_dag*P_j_dag)).simplify().eval_vev().simplify()
RDM2 = RDM2.subst_all(zero_term)

In [30]:
print(RDM2.latex())

u_{p}^{2}  u_{q}^{2}  z^{(2)}_{p,q}  -  \delta_{p q} u_{p}^{2}  v_{p}^{2}   + u_{p}^{2}  u_{q}  v_{q}  z^{(1)}_{p}   + u_{q}^{2}  u_{p}  v_{p}  z^{(1)}_{q}   + u_{p}  u_{q}  v_{p}  v_{q}  -  2 \delta_{p q} u_{p}^{3}  v_{p}  z^{(1)}_{p} 


In [ ]:
L1 = IndexedBase('L1')
L2 = IndexedBase('L2')

expr1 = dr.define(L1[p],dL_dt_1)
 
 
Res2 = IndexedBase('Res2')
expr2 = dr.define(L2[p,q],dL_dt_2)
 

eval_equ0 = optimize(
    [expr1,expr2],
    interm_fmt='tau{}',drop_cutoff=2
)
print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_Z.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")
